# 07. Mapping
---

We will use this notebook to plot out the results of our efforts, in order to have an easy to interpret visual representation of all the data gathered.

---
## Table of Contents
---


 - [Import Resources](#Import-Resources)
 - [Setting Up Data for Mapping](#Setting-Up-Data-for-Mapping)
 - [Create and Save Map](#Create-and-Save-Map)

---
### Import Resources
---

Here we import the necessary libraries to enable mapping.  Furthermore, we read-in the cleaned `geodata` data.

In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, FeatureGroupSubGroup

Read-in and light cleaning of csv

In [2]:
geo_data = pd.read_csv("../datasets/geodata.csv")

# Drop unnecessary columns
geo_data.drop(columns = "Unnamed: 0", inplace = True)

geo_data.head()

,username,tweet,date_posted,expanded_tweet,highway,streets,exits,directions,locs,lanes,relocs,town,location_lookup,lat,lon
0,511nji295,Crash on I-295 southbound South of Exit 29 - U...,2019-11-06 23:56:56+00:00,Crash on Interstate 295 southbound South of Ex...,Interstate 295,NaN,Exit 29 - US-30,southbound,NaN,right lane,South of,Haddon Heights,"Haddon Heights, NJ",39.881870,-75.059300
1,511njace,"Construction, bridge painting on Atlantic City...",2019-11-06 23:52:57+00:00,"Construction, bridge painting on Atlantic City...",Atlantic City Expressway,NaN,"Exit 12 - US-40 - Wrangleboro Road, Exit 12 - ...",westbound,NaN,right lane,"between East of, West of","Hamilton Township, Hamilton Township","Hamilton Township, Hamilton Township, NJ",40.230360,-74.723860
2,511njtpk,Roadwork on New Jersey Turnpike inner roadway ...,2019-11-06 23:41:56+00:00,Roadwork on New Jersey Turnpike inner roadway ...,"New Jersey Turnpike, Interchange 6 - Pennsylva...",NaN,NaN,northbound,NaN,NaN,from,NaN,"New Jersey Turnpike, Interchange 6 - Pennsylva...",41.022837,-78.444277
3,511nji76,Crash on I-76 eastbound at Exit 2 - I-676 (Cam...,2019-11-06 23:41:56+00:00,Crash on Interstate 76 eastbound at Exit 2 - I...,Interstate 76,NaN,Exit 2 - Interstate 676,eastbound,NaN,right shoulder,NaN,Camden,"Camden, NJ",39.945250,-75.119130
4,511njtpk,Roadwork on New Jersey Turnpike inner roadway ...,2019-11-06 23:27:56+00:00,Roadwork on New Jersey Turnpike inner roadway ...,"New Jersey Turnpike, Interchange 14 - Intersta...",NaN,NaN,Southbound,NaN,all lanes,"between North of, and South of",Newark,"Newark, NJ",40.731970,-74.174210


---
### Setting up Data for Mapping
---

In [3]:
# Create a list of usernames for dictionary reference
usernames = list(set(geo_data['username']))

In [4]:
user_dict = {}

for user in usernames:
    user_dict[user] = geo_data[geo_data['username'] == user].reset_index(drop = True)

Building a color dictionary to assign colors to respective twitter accounts that appear on html map.

In [5]:
## Color dictionary for usernames
color_dict = {
    'NJTRANSIT_PVL' : 'black',
    '511njace' : 'gray',
    '511njgsp' : 'silver',
    'DRPA_PAandNJ' : 'whitesmoke',
    'NJTRANSIT_NBUS' : 'rosybrown',
    '511nji78' : 'firebrick',
    'NJTRANSIT_MOBO' : 'red',
    'NJTRANSIT_NJCL' : 'darksalmon',
    'NJTRANSIT_ME' : 'sienna',
    '511nji676' : 'sandybrown',
    '511nji287' : 'bisque',
    'NJTRANSIT_MBPJ' : 'tan',
    '511nji280' : 'moccasin',
    '511nj24' : 'floralwhite',
    '511nji195' : 'gold',
    '511nj42' : 'darkkhaki',
    '511njtpk' : 'lightgoldenrodyellow',
    'NJTRANSIT' : 'olivedrab',
    'NJTRANSIT_SBUS' : 'chartreuse',
    'NJTRANSIT_RVL' : 'palegreen',
    'NJTurnpike' : 'darkgreen',
    'demembridge' : 'seagreen',
    'NJDOT_info' : 'mediumspringgreen',
    '511nji295' : 'lightseagreen',
    'NJTRANSIT_NEC' : 'paleturquoise',
    'NJTRANSIT_HBLR' : 'darkcyan',
    '511nj55' : 'deepskyblue',
    '511nj495' : 'aliceblue',
    'NJTRANSIT_RL' : 'navy',
    'NJTRANSIT_NLR' : 'blue',
    '511nji80' : 'plum',
    '511njbt' : 'm',
    '511nji76' : 'palevioletred'
}

## Add color column to each user dataframe with the color for that user
for user in user_dict:
    user_dict[user]['color'] = color_dict[user]

---
### Create and Save Map
---

In [6]:
## Adapted from Noah Chirstiansen's code

## Instantiate our map object
my_map = folium.Map(location = [40.717700, -74.043110], ## initial map focus
                    zoom_start = 10, ## zoom level
                    control_scale=True, 
                    prefer_canvas = True)

## Create the html for our popups
def popup_url(point):
    # Add username
    popup_html  = f'<h3>{point["username"]}</h3><br>'
    # Add tweet
    popup_html += f'<b>Description:</b> {point["tweet"]} <br>'
    # Add date posted
    popup_html += f'<b>Date/Time:</b> {point["date_posted"]} <br>'

    return popup_html

## Function to add a point to our map
def add_point(point):
    ## Parse the html we generate with the 'popup_url' function
    html_string = folium.Html(popup_url(point), script=True)
    
    ## Add a circle marker for the point
    folium.vector_layers.CircleMarker(location=[point['lat'], point['lon']],
                        color = point['color'], #set color using the color column
                        fill = True,             
                        fill_color = point['color'],
                        fill_opacity = 0.7,
                        radius=7,
                        weight=2,
                        popup = folium.Popup(html_string,max_width=500, parse_html=True), ## add our popup text to the point
                        control = True,
                        overlay = True).add_to(sub_group) ## add to the subgroup layer for that point

for user in user_dict:
    ## Instantiate MarkerCluster for each layer (this will cluster points together at higher zoom levels)
    marker_cluster = MarkerCluster(control = False)

    ## Add MarkerCluster to our map
    marker_cluster.add_to(my_map)

    ## Instantiate subgroup for each layer (this will group together all points in each dataframe)
    sub_group = FeatureGroupSubGroup(marker_cluster, name = user, control= True, show = True)

    ## Use apply to loop through our dataframe and add each point to our subgroup layer
    user_dict[user].apply(add_point, axis = 1)

    ## Add subgroup layer to our map
    sub_group.add_to(my_map)

## Add LayerControl to our map (this will allow us to select layers to display on our map)
folium.map.LayerControl(collapsed=False).add_to(my_map)

## Save map to html file
my_map.save('../maps/my_map.html')

---